# **Bill Interpreter — Indian Expense Report Automation**

This notebook demonstrates how to build an end-to-end pipeline for extracting structured data from Indian bills and receipts using Sarvam AI's suite of models.

### **Use Case**
Automate expense reporting for Indian businesses and individuals.
1. **Extract:** Use **Sarvam Vision Document Intelligence** to OCR the bill image or PDF.
2. **Parse:** Use **Sarvam-M** to convert raw OCR text into clean, structured JSON.
3. **Export:** Use **openpyxl** to write all bill data into a formatted Excel expense report.

### **Supported Formats**
- Images: `.jpg`, `.jpeg`, `.png`
- Documents: `.pdf`
- Languages: 22 Indian languages + English (Hindi, Tamil, Telugu, Kannada, Bengali, Gujarati, Marathi, and more)

In [ ]:
# Pinning versions for reproducibility
!pip install -Uqq sarvamai==0.1.24 openpyxl>=3.1.0 python-dotenv>=1.0.0 Pillow>=10.3.0

### **1. Setup & API Key**

Obtain your API key from the [Sarvam AI Dashboard](https://dashboard.sarvam.ai).
Create a `.env` file in this directory with `SARVAM_API_KEY=your_key_here`, or set the environment variable directly.

In [ ]:
import os
import json
import re
import zipfile
import tempfile
from pathlib import Path

from dotenv import load_dotenv
from sarvamai import SarvamAI

load_dotenv()

SARVAM_API_KEY = os.environ.get("SARVAM_API_KEY", "")
if not SARVAM_API_KEY or SARVAM_API_KEY == "YOUR_SARVAM_API_KEY":
    raise RuntimeError(
        "SARVAM_API_KEY is not set. Add it to your .env file or set the environment variable."
    )

client = SarvamAI(api_subscription_key=SARVAM_API_KEY)

print("Client initialised.")

### **2. Step 1 — EXTRACT: Document Intelligence Helper**

`extract_bill_text` sends the bill file to Sarvam Vision Document Intelligence and returns the extracted text as a Markdown string.

The API uses an async job workflow: create → upload → start → wait → download (ZIP) → unzip.

> **Note:** The API accepts `.pdf` or `.zip` only. PNG/JPG images are automatically wrapped in a ZIP before upload.

In [ ]:
_IMAGE_EXTENSIONS = {'.jpg', '.jpeg', '.png'}


def extract_bill_text(file_path: str, language: str = "en-IN") -> str:
    """Extract text from a bill image or PDF using Sarvam Document Intelligence.

    Images (.jpg, .png) are automatically wrapped in a ZIP archive before upload,
    as the API only accepts PDF or ZIP files directly.
    """
    path = Path(file_path)
    upload_path = file_path
    tmp_zip: str | None = None

    if path.suffix.lower() in _IMAGE_EXTENSIONS:
        # Wrap image in a flat ZIP — required by the Document Intelligence API
        with tempfile.NamedTemporaryFile(suffix='.zip', delete=False) as tmp:
            tmp_zip = tmp.name
        with zipfile.ZipFile(tmp_zip, 'w', zipfile.ZIP_DEFLATED) as zf:
            zf.write(file_path, arcname=path.name)
        upload_path = tmp_zip

    try:
        job = client.document_intelligence.create_job(
            language=language,
            output_format="md"
        )
        job.upload_file(upload_path)
        job.start()

        status = job.wait_until_complete()
        if status.job_state != "Completed":
            raise RuntimeError(
                f"Document Intelligence job ended with state: {status.job_state}. "
                f"Details: {status}"
            )

        # Download ZIP output to a temp file, then extract markdown in-memory
        with tempfile.NamedTemporaryFile(suffix='.zip', delete=False) as tmp:
            out_zip = tmp.name

        try:
            job.download_output(out_zip)
            with zipfile.ZipFile(out_zip, 'r') as zf:
                md_files = [f for f in zf.namelist() if f.endswith('.md')]
                if not md_files:
                    raise RuntimeError(
                        "No markdown output found in Document Intelligence result. "
                        f"ZIP contents: {zf.namelist()}"
                    )
                with zf.open(md_files[0]) as f:
                    return f.read().decode('utf-8')
        finally:
            os.unlink(out_zip)

    finally:
        if tmp_zip:
            os.unlink(tmp_zip)


print("extract_bill_text defined.")

### **3. Step 2 — PARSE: Structured JSON Extraction**

`parse_bill` sends the raw OCR markdown to **Sarvam-M** with a structured prompt and returns a validated Python dict.

A `confidence` score below **0.85** triggers a warning — flag those bills for manual review.

In [ ]:
PARSE_SYSTEM_PROMPT = """You are a precise Indian bill data extractor. Extract the following fields from the bill text and return ONLY valid JSON with no other text, no markdown fences, no explanation.

Required JSON schema:
{
  \"vendor_name\": \"<string>\",
  \"vendor_gstin\": \"<string or null>\",
  \"date\": \"<DD-MM-YYYY format>\",
  \"invoice_number\": \"<string or null>\",
  \"line_items\": [{\"description\": \"<str>\", \"quantity\": <float>, \"unit_price\": <float>, \"total\": <float>}],
  \"subtotal\": <float>,
  \"cgst\": <float or null>,
  \"sgst\": <float or null>,
  \"igst\": <float or null>,
  \"total_amount\": <float>,
  \"currency\": \"<INR or other>\",
  \"language_detected\": \"<primary language of the bill, e.g. English, Hindi, Tamil>\",
  \"confidence\": <float between 0.0 and 1.0>
}

Rules:
- Use null (not \\\"null\\\") for fields not present in the bill
- All monetary values must be float, not string
- confidence reflects how completely all fields could be read (1.0 = perfect, 0.0 = unreadable)
- Return ONLY the JSON object"""


def parse_bill(raw_text: str) -> dict:
    """Parse raw bill text into structured JSON using Sarvam-M."""
    response = client.chat.completions(
        messages=[
            {"role": "system", "content": PARSE_SYSTEM_PROMPT},
            {"role": "user", "content": f"Extract data from this bill:\n\n{raw_text}"}
        ]
    )

    if not response or not response.choices:
        raise ValueError("Sarvam-M returned no response. Check your API quota.")

    raw_json = response.choices[0].message.content.strip()
    # Strip markdown code fences if the model wraps output anyway
    raw_json = re.sub(r'^```(?:json)?\s*|\s*```$', '', raw_json, flags=re.DOTALL).strip()

    try:
        parsed = json.loads(raw_json)
    except json.JSONDecodeError:
        print(f"ERROR: Could not parse JSON from model response:\n{raw_json}")
        raise

    confidence = parsed.get("confidence", 1.0)
    if confidence < 0.85:
        print(f"WARNING: Low confidence ({confidence:.2f}) \u2014 review this bill manually before submitting the expense report.")

    return parsed


print("parse_bill defined.")

### **4. Step 3 — EXPORT: Excel Expense Report Writer**

`write_to_excel` takes a list of parsed bill dicts and writes them into a formatted `.xlsx` file using openpyxl.

- If the file already exists, new rows are appended (batch processing).
- Line items are serialised as a readable string per row.
- Column widths are auto-sized.

In [ ]:
from openpyxl import Workbook, load_workbook
from openpyxl.styles import Font, PatternFill, Alignment

_COLUMNS = [
    "vendor_name", "vendor_gstin", "date", "invoice_number",
    "subtotal", "cgst", "sgst", "igst", "total_amount",
    "currency", "language_detected", "confidence", "line_items",
]

_HEADER_LABELS = {
    "vendor_name":       "Vendor Name",
    "vendor_gstin":      "GSTIN",
    "date":              "Date",
    "invoice_number":    "Invoice No.",
    "subtotal":          "Subtotal (₹)",
    "cgst":              "CGST (₹)",
    "sgst":              "SGST (₹)",
    "igst":              "IGST (₹)",
    "total_amount":      "Total (₹)",
    "currency":          "Currency",
    "language_detected": "Language",
    "confidence":        "Confidence",
    "line_items":        "Line Items",
}


def _format_line_items(items: list) -> str:
    if not items:
        return ""
    parts = [
        f"{i.get('description', '')} x{i.get('quantity', 1)} "
        f"@ ₹{i.get('unit_price', 0)} = ₹{i.get('total', 0)}"
        for i in items
    ]
    return " | ".join(parts)


def write_to_excel(bills: list, output_path: str) -> None:
    """Write a list of parsed bill dicts to a formatted Excel expense report."""
    path = Path(output_path)

    if path.exists():
        wb = load_workbook(path)
        ws = wb["Expense Report"]
    else:
        wb = Workbook()
        ws = wb.active
        ws.title = "Expense Report"

        header_fill = PatternFill(start_color="1F4E79", end_color="1F4E79", fill_type="solid")
        header_font = Font(color="FFFFFF", bold=True)

        for col_idx, col_key in enumerate(_COLUMNS, start=1):
            cell = ws.cell(row=1, column=col_idx, value=_HEADER_LABELS[col_key])
            cell.font = header_font
            cell.fill = header_fill
            cell.alignment = Alignment(horizontal="center")

        ws.freeze_panes = "A2"

    for bill in bills:
        row = []
        for col_key in _COLUMNS:
            if col_key == "line_items":
                row.append(_format_line_items(bill.get("line_items", [])))
            else:
                row.append(bill.get(col_key))
        ws.append(row)

    for col in ws.columns:
        max_len = max((len(str(cell.value or "")) for cell in col), default=10)
        ws.column_dimensions[col[0].column_letter].width = min(max_len + 4, 50)

    wb.save(output_path)
    print(f"Expense report saved to: {output_path}")


print("write_to_excel defined.")

### **5. End-to-End Pipeline**

`process_bills` ties all three steps together. Pass a list of file paths (images or PDFs) and an output Excel path. Failed files are logged and skipped — processing continues for remaining bills.

In [ ]:
def process_bills(file_paths: list, output_path: str = "expense_report.xlsx") -> list:
    """
    End-to-end pipeline: extract -> parse -> export for one or more bill files.

    Args:
        file_paths: List of paths to bill images (.jpg, .png) or PDFs (.pdf).
        output_path: Path for the output Excel file. Appends if file exists.

    Returns:
        List of successfully parsed bill dicts.
    """
    results = []

    for i, file_path in enumerate(file_paths, start=1):
        print(f"\n[{i}/{len(file_paths)}] Processing: {file_path}")
        try:
            print("  Step 1/3 \u2014 Extracting text via Document Intelligence...")
            raw_text = extract_bill_text(file_path)

            print("  Step 2/3 \u2014 Parsing structured data with Sarvam-M...")
            parsed = parse_bill(raw_text)

            results.append(parsed)
            print(
                f"  Vendor: {parsed.get('vendor_name')} | "
                f"Total: \u20b9{parsed.get('total_amount')} | "
                f"Confidence: {parsed.get('confidence', 0):.2f}"
            )

        except Exception as e:
            print(f"  ERROR: Failed to process {file_path}: {e}")

    if results:
        print(f"\nStep 3/3 \u2014 Writing {len(results)} bill(s) to Excel...")
        write_to_excel(results, output_path)
    else:
        print("\nNo bills successfully processed \u2014 Excel report not written.")

    return results


print("process_bills defined.")

### **6. Demo \u2014 Run the Pipeline**

Cell 8 creates a synthetic Indian GST invoice using Pillow (no real bill required), then runs the full pipeline. The bill mimics a typical kirana store invoice with GSTIN, line items, CGST, and SGST.

In [ ]:
from PIL import Image, ImageDraw, ImageFont


def _create_sample_bill(output_path: str = "sample_data/sample_bill.png") -> str:
    """Create a synthetic Indian GST invoice image for demo purposes."""
    Path(output_path).parent.mkdir(parents=True, exist_ok=True)

    img = Image.new("RGB", (620, 820), color="white")
    draw = ImageDraw.Draw(img)

    # Try to load a system font; fall back gracefully
    try:
        font_bold  = ImageFont.truetype("/System/Library/Fonts/Helvetica.ttc", 18)
        font_reg   = ImageFont.truetype("/System/Library/Fonts/Helvetica.ttc", 14)
        font_small = ImageFont.truetype("/System/Library/Fonts/Helvetica.ttc", 12)
    except (IOError, OSError):
        font_bold = font_reg = font_small = ImageFont.load_default()

    y = 40
    draw.text((310, y), "TAX INVOICE", font=font_bold, fill="black", anchor="mm")
    y += 40
    draw.line([(40, y), (580, y)], fill="black", width=2)
    y += 20

    # Vendor block
    draw.text((40, y), "Sharma General Stores",           font=font_bold, fill="black")
    y += 26
    draw.text((40, y), "123 MG Road, Bengaluru - 560001", font=font_reg,  fill="black")
    y += 22
    draw.text((40, y), "GSTIN: 29AABCS1429B1Z1",          font=font_reg,  fill="black")
    y += 22
    draw.text((40, y), "Phone: +91-80-12345678",          font=font_reg,  fill="black")
    y += 30
    draw.line([(40, y), (580, y)], fill="black", width=1)
    y += 20

    # Invoice meta
    draw.text((40,  y), "Invoice No: INV-2025-0042", font=font_reg, fill="black")
    draw.text((380, y), "Date: 15-01-2025",          font=font_reg, fill="black")
    y += 40

    # Table header
    draw.rectangle([(40, y), (580, y + 26)], fill="#CCCCCC")
    for x, label in [(45, "Description"), (330, "Qty"), (400, "Unit Price"), (500, "Total")]:
        draw.text((x, y + 5), label, font=font_small, fill="black")
    y += 30

    line_items = [
        ("Basmati Rice (5 kg)",       "2", "350.00",  "700.00"),
        ("Refined Oil (1 L)",         "3", "180.00",  "540.00"),
        ("Toor Dal (1 kg)",           "2", "140.00",  "280.00"),
        ("Whole Wheat Atta (10 kg)",  "1", "450.00",  "450.00"),
    ]
    for desc, qty, price, total in line_items:
        draw.text((45,  y), desc,           font=font_small, fill="black")
        draw.text((330, y), qty,            font=font_small, fill="black")
        draw.text((400, y), f"Rs.{price}",  font=font_small, fill="black")
        draw.text((500, y), f"Rs.{total}",  font=font_small, fill="black")
        y += 26

    y += 10
    draw.line([(40, y), (580, y)], fill="black", width=1)
    y += 15

    for label, value in [
        ("Subtotal:",   "Rs.1970.00"),
        ("CGST @4.5%:", "Rs.88.65"),
        ("SGST @4.5%:", "Rs.88.65"),
    ]:
        draw.text((360, y), label, font=font_reg,   fill="black")
        draw.text((500, y), value, font=font_small, fill="black")
        y += 26

    draw.line([(350, y), (580, y)], fill="black", width=2)
    y += 10
    draw.text((360, y), "TOTAL:",      font=font_bold, fill="black")
    draw.text((500, y), "Rs.2147.30",  font=font_bold, fill="black")

    draw.text((40, 790), "Thank you for your business!", font=font_small, fill="gray")

    img.save(output_path)
    print(f"Sample bill created: {output_path}")
    return output_path


# --- Run the demo ---
sample_path = _create_sample_bill()
results = process_bills([sample_path], output_path="expense_report.xlsx")

### **7. Results**

View the parsed JSON and download the generated Excel report.

In [ ]:
from IPython.display import FileLink, display

if results:
    print("=== Parsed Bill Data (first bill) ===\n")
    print(json.dumps(results[0], indent=2, ensure_ascii=False))

    print("\n=== Download Expense Report ===")
    display(FileLink("expense_report.xlsx", result_html_prefix="Click to download: "))
else:
    print("No results to display. Check the error messages above.")

### **8. Error Reference**

| Error Code | HTTP Status | Cause | Solution |
| :--- | :--- | :--- | :--- |
| `invalid_api_key_error` | 403 | Invalid API key | Verify your key at [dashboard.sarvam.ai](https://dashboard.sarvam.ai). |
| `insufficient_quota_error` | 429 | Quota exceeded | Check your usage limits. |
| `internal_server_error` | 500 | Server-side issue | Wait and retry the request. |
| Job state not `Completed` | — | Doc Intelligence failure | Check file format; supported: `.pdf`, `.zip` (images auto-wrapped). |
| `JSONDecodeError` | — | Sarvam-M returned non-JSON | Usually transient; re-run the cell. |

### **9. Conclusion & Resources**

This recipe demonstrates how to chain **Sarvam Vision** and **Sarvam-M** into a practical expense automation workflow — processing bills in any Indian language with a single pipeline call.

* [Sarvam AI Docs](https://docs.sarvam.ai)
* [Document Intelligence API](https://docs.sarvam.ai/api-reference-docs/document-intelligence)
* [Sarvam-M Chat API](https://docs.sarvam.ai/api-reference-docs/chat)
* [Indic Language Support](https://docs.sarvam.ai/language-support)

**Keep Building!**